In [ ]:
def get_jwt():
    payload = {
            "client_id": "e-money-portal",
            "grant_type": "password",
            "username": os.environ["AFRICA_METABASE_USERNAME"],
            "password": os.environ["AFRICA_METABASE_PASSWORD"]
    }

    r = requests.post(
        "https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data=payload
    )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']

    return jwt_access_token


def activate_retailers():
    initialize_env()

    query = '''
        with  recent_deactivation as(
            SELECT DISTINCT main_system_id 
            FROM maxab_logs.emoney_retailers 
            WHERE status_changed_FROM = 'APPROVED' AND status_changed_to = 'NOT_IN_THE_SCOPE'
            AND extract(MONTH, REGEXP_SUBSTR(changed_at, '[0-9, -]+', 1)::DATE ) >= extract(MONTH, CURRENT_DATE - INTERVAL '2 MONTH')
        )

        SELECT DISTINCT r1.id AS main_system_id
        FROM public.retailers r1 
        LEFT JOIN emoney.retailers r2 ON r1.id = r2.main_system_id 
        LEFT JOIN recent_deactivation ON r1.id = recent_deactivation.main_system_id
        WHERE
            (r2.main_system_id IS NULL OR r2.status = 'NOT_IN_THE_SCOPE')
            AND DATE_TRUNC('MONTH', r1.created_at) >= CAST(CURRENT_DATE - INTERVAL '2 MONTH' AS DATE) 
            AND recent_deactivation.main_system_id IS NULL

    '''

    df = snowflake_query(query)

    # data cleaning:
    df.main_system_id = pd.to_numeric(df.main_system_id, errors='coerce').astype('int')
    main_system_ids = df['main_system_id'].values.tolist()

    # get jwt
    jwt_access_token = get_jwt()

    # headers for the request.
    headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": "Bearer %s" % jwt_access_token,
        }

    payload = {
        "status": "APPROVED",
        "retailersMainSystemIds": main_system_ids
    }

    response = requests.put(
        "https://api.maxabma.com/emoney/api/v1/retailers/status",
        headers=headers,
        data=json.dumps(payload)
    )

    if response.status_code == 200:
        print("Retailer status is updated")
    else:
        print("Error in updating retailer status")